In [1]:
from openagentkit.modules.openai import OpenAIExecutor
from openagentkit.utils.tool_wrapper import tool
from pydantic import BaseModel
from typing import Annotated
import openai
import os

In [2]:
# Define a tool
@tool(
    description="Get the weather of a city", # Define the tool description
)
def get_weather(city: Annotated[str, "The city to get the weather of"]) -> str: # Each argument must be of type Annotated
    """Get the weather of a city"""

    # Actual implementation here...
    # ...

    # Define a response schema
    class WeatherResponse(BaseModel):
        city: str
        weather: str
        temperature: float
        feels_like: float
        humidity: float

    return WeatherResponse( # The Response must be a pydantic model
        city=city,
        weather="sunny",
        temperature=20,
        feels_like=22,
        humidity=0.5,
    )

In [3]:
# Initialize OpenAI client
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Initialize LLM service
executor = OpenAIExecutor(
    client=client,
    model="gpt-4o-mini",
    system_message="""
    You are a helpful assistant that can answer questions and help with tasks.
    You are also able to use tools to get information.
    """,
    tools=[get_weather],
    temperature=0.5,
    max_tokens=100,
    top_p=1.0,
)

generator = executor.execute(
    messages=[
        {"role": "user", "content": "What's the weather like in New York?"}
    ]
)

for response in generator:
    print(response.content)

2025-04-15 18:51:48.103 | DEBUG    | openagentkit.modules.openai.openai_executor:execute:129 - Context: [{'role': 'system', 'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-04-15 18:51:48\n\n        \n        '}, {'role': 'user', 'content': "What's the weather like in New York?"}]
2025-04-15 18:51:49.909 | INFO     | openagentkit.modules.openai.openai_executor:execute:144 - Response Received: role='assistant' content=None tool_calls=[{'id': 'call_UhqLA87MclqstuUEBu7Kt0zJ', 'type': 'function', 'function': {'arguments': '{"city":"New York"}', 'name': 'get_weather'}}] tool_results=None refusal=None audio=None usage=UsageResponse(prompt_tokens=112, completion_tokens=16, total_tokens=128, prompt_tokens_details=PromptTokensDetails(cached_tokens=0, audio_tokens=0), completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, ac

The weather in New York is currently sunny, with a temperature of 20°C (feels like 22°C) and humidity at 50%.
